In [1]:
from fast_to_SQL.fast_to_SQL import to_sql_fast
import pandas as pd
import numpy as np
from urllib.parse import quote_plus
import sqlalchemy

params = quote_plus(
    """DRIVER={ODBC Driver 13 for SQL Server};
    SERVER=server-dw01.database.windows.net;
    DATABASE=DataWarehouseStaging;
    UID=dw_userlogin;
    PWD=kljDueK!7"""
)

ENGINE = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)

In [2]:
def dedupe(df, dupe_df):
    comb_df = df.merge(dupe_df, how = 'left', left_on = 'com_id_h', right_on = 'Old')
    comb_df['deduped_id'] = comb_df['Actual_New'].combine_first(comb_df['com_id_h'])
    agg_cols = [i for i in comb_df.columns if i[:2] == '20']
    agg_df = comb_df.groupby('deduped_id')[agg_cols].mean()
    agg_df = agg_df.reset_index()
    agg_df = agg_df.rename(columns = {'deduped_id': 'com_id_h'})
    return agg_df

In [3]:
dupe_df = pd.read_sql('''SELECT *
                    FROM marketing.Z_WP_OldtoNewIDs''', ENGINE)

In [4]:
df_TO = pd.read_sql('''SELECT *
                    FROM pam.TO_dataset''', ENGINE)

In [5]:
df_TO_deduped = dedupe(df_TO, dupe_df)

In [6]:
to_sql_fast(df_TO_deduped, name = 'pam.TO_dataset_dedupe', engine = ENGINE, if_exists = 'replace')

In [7]:
df_trans = pd.read_sql('''SELECT *
                    FROM pam.trans_dataset''', ENGINE)

In [8]:
df_trans_deduped = dedupe(df_trans, dupe_df)

In [9]:
to_sql_fast(df_trans_deduped, name = 'pam.trans_dataset_dedupe', engine = ENGINE, if_exists = 'replace')

In [11]:
df_feature = pd.read_sql('''SELECT * FROM pam.dataset_features''', ENGINE)

In [14]:
comb_df_feature = df_feature.merge(dupe_df, how = 'left', left_on = 'com_id_h', right_on = 'Old')

In [16]:
comb_df_feature['deduped_id'] = comb_df_feature['Actual_New'].combine_first(comb_df_feature['com_id_h'])

In [19]:
comb_df_feature.columns

Index(['com_id_h', 'file_date', 'MCC_DESC', 'tenure_in_months', 'Old',
       'Actual_New', 'Id_Conv_type', 'deduped_id'],
      dtype='object')

In [20]:
df_feature_dedupe = comb_df_feature[['deduped_id', 'file_date', 'MCC_DESC', 'tenure_in_months']]

In [21]:
df_feature_dedupe = df_feature_dedupe.rename(columns = {'deduped_id': 'com_id_h'})

In [23]:
to_sql_fast(df_feature_dedupe, name = 'pam.dataset_features_dedupe', engine = ENGINE, if_exists = 'replace')